<a href="https://colab.research.google.com/github/YasuharaSky/LIGN167_Emotion_Transfer/blob/main/Copy_of_LIGN167_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LIGN167 Project

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


data = pd.read_csv('/content/drive/My Drive/combined_emotions.csv')

## Import data from GoEmotion

## Train-Validation-Test Split for task 1& task 2

In [ ]:
import pandas as pd
import os

In [ ]:
def split_dataset(csv_path, train_size=0.7, val_size=0.15, test_size=0.15):
  df = pd.read_csv(csv_path)

  train_val, test = train_test_split(
    df,
    test_size=test_size,
    stratify=df['emotion'],
    random_state=42
  )

  relative_val_size = val_size / (train_size + val_size)
  train, val = train_test_split(
    train_val,
    test_size=relative_val_size,
    stratify=train_val['emotion'],
    random_state=42
  )

  train.to_csv('/content/drive/My Drive/train_data.csv', index=False)
  val.to_csv('/content/drive/My Drive/val_data.csv', index=False)
  test.to_csv('/content/drive/My Drive/test_data.csv', index=False)

  return train, val, test

In [ ]:
train, val, test = split_dataset('/content/drive/My Drive/combined_emotions.csv')

## Step 1. Sentiment Analysis

In [ ]:
from transformers import AutoTokenizer

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2, stratify=data['emotion'])
val, test = train_test_split(test, test_size=0.5, stratify=test['emotion'])

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
train['emotion_label'] = label_encoder.fit_transform(train['emotion'])
val['emotion_label'] = label_encoder.transform(val['emotion'])
test['emotion_label'] = label_encoder.transform(test['emotion'])

In [ ]:
emotion_classes = dict(zip(range(len(label_encoder.classes_)), label_encoder.classes_))
print(emotion_classes)

{0: 'anger', 1: 'caring', 2: 'confusion', 3: 'fear', 4: 'joy', 5: 'sadness'}


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_data(data):
    return tokenizer(list(data['text']), padding=True, truncation=True, max_length=128, return_tensors="pt")

train_encodings = tokenize_data(train)
val_encodings = tokenize_data(val)
test_encodings = tokenize_data(test)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=6)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
def train_epoch(model, train_loader, optimizer, loss_fn, device):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc="Training"):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]

        # forward
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs.logits, labels)

        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(train_loader)

def validate_epoch(model, val_loader, loss_fn, device):
    model.eval()
    total_loss = 0
    correct = 0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validating"):
            input_ids, attention_mask, labels = [b.to(device) for b in batch]

            # forward
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = loss_fn(outputs.logits, labels)
            total_loss += loss.item()

            # accuracy
            preds = torch.argmax(outputs.logits, dim=1)
            correct += (preds == labels).sum().item()
    accuracy = correct / len(val_loader.dataset)
    return total_loss / len(val_loader), accuracy


In [ ]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
from transformers import AutoTokenizer
import torch
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(train['emotion_label'].values))
val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], torch.tensor(val['emotion_label'].values))

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)


In [ ]:
!pip install tqdm  # Install tqdm if you haven't already
from tqdm import tqdm  # Import tqdm into your script

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 5
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    # train
    train_loss = train_epoch(model, train_loader, optimizer, loss_fn, device)
    print(f"Training Loss: {train_loss:.4f}")

    # valid
    val_loss, val_accuracy = validate_epoch(model, val_loader, loss_fn, device)
    print(f"Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}")


Epoch 1/5


Training: 100%|██████████| 89/89 [00:09<00:00,  9.37it/s]


Training Loss: 0.4369


Validating: 100%|██████████| 12/12 [00:00<00:00, 42.45it/s]


Validation Loss: 0.0499, Accuracy: 0.9860
Epoch 2/5


...

Validation Loss: 0.0410, Accuracy: 0.9916
Epoch 5/5


Training: 100%|██████████| 89/89 [00:07<00:00, 11.63it/s]


Training Loss: 0.0030


Validating: 100%|██████████| 12/12 [00:00<00:00, 56.14it/s]

Validation Loss: 0.0369, Accuracy: 0.9916


In [ ]:
model.save_pretrained("emotion_classifier")
tokenizer.save_pretrained("emotion_classifier")

('emotion_classifier/tokenizer_config.json',
 'emotion_classifier/special_tokens_map.json',
 'emotion_classifier/vocab.txt',
 'emotion_classifier/added_tokens.json',
 'emotion_classifier/tokenizer.json')

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


tokenizer = AutoTokenizer.from_pretrained("emotion_classifier")
model = AutoModelForSequenceClassification.from_pretrained("emotion_classifier")
model.to(device)


emotion_classes = {0: 'anger', 1: 'caring', 2: 'confusion', 3: 'fear', 4: 'joy', 5: 'sadness'}

test_sentence = "I don't know how to do the homework"
inputs = tokenizer(test_sentence, return_tensors="pt", padding=True, truncation=True)
inputs = {key: val.to(device) for key, val in inputs.items()}

outputs = model(**inputs)
predicted_class = torch.argmax(outputs.logits, dim=1).item()

print(f"Sentence: {test_sentence}")
print(f"Predicted Emotion Class: {predicted_class} ({emotion_classes[predicted_class]})")


Sentence: I don't know how to do the homework
Predicted Emotion Class: 3 (fear)


## STEP 2: Regression (Level Estimate)

In [ ]:
max_degree = train['intensity'].max()
train['intensity'] = train['intensity'] / max_degree
val['intensity'] = val['intensity'] / max_degree
test['intensity'] = test['intensity'] / max_degree

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_data(data, tokenizer):
    encodings = tokenizer(list(data['text']), padding=True, truncation=True, max_length=128, return_tensors="pt")
    labels = torch.tensor(data['intensity'].tolist(), dtype=torch.float)
    return TensorDataset(encodings['input_ids'], encodings['attention_mask'], labels)

train_dataset = tokenize_data(train, tokenizer)
val_dataset = tokenize_data(val, tokenizer)
test_dataset = tokenize_data(test, tokenizer)

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

In [ ]:
from transformers import AutoModel
import torch.nn as nn

class EmotionRegressor(nn.Module):
    def __init__(self, model_name):
        super(EmotionRegressor, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.regressor = nn.Linear(self.bert.config.hidden_size, 1)
        self.dropout = nn.Dropout(0.1)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.regressor(pooled_output)
        return logits

In [ ]:
class EmotionRegressor(nn.Module):
    def __init__(self, model_name):
        super(EmotionRegressor, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.regressor = nn.Linear(self.bert.config.hidden_size, 1)
        self.dropout = nn.Dropout(0.1)

    def forward(self, input_ids, attention_mask, token_type_ids=None, **kwargs):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.regressor(pooled_output)
        return logits

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EmotionRegressor("bert-base-uncased")
model.to(device)

EmotionRegressor(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementw

In [ ]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = nn.MSELoss()

In [ ]:
from tqdm import tqdm

def train_epoch(model, train_loader, optimizer, loss_fn, device):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc="Training"):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]

        # forward
        outputs = model(input_ids=input_ids, attention_mask=attention_mask).squeeze()
        loss = loss_fn(outputs, labels)

        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(train_loader)


In [ ]:
def validate_epoch(model, val_loader, loss_fn, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validating"):
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask).squeeze()
            loss = loss_fn(outputs, labels)
            total_loss += loss.item()
    return total_loss / len(val_loader)

In [ ]:
epochs = 7
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")


    train_loss = train_epoch(model, train_loader, optimizer, loss_fn, device)
    print(f"Training Loss: {train_loss:.4f}")


    val_loss = validate_epoch(model, val_loader, loss_fn, device)
    print(f"Validation Loss: {val_loss:.4f}")


Epoch 1/7


Training: 100%|██████████| 178/178 [00:09<00:00, 19.10it/s]


Training Loss: 0.0431


Validating: 100%|██████████| 23/23 [00:00<00:00, 86.94it/s]


Validation Loss: 0.0129
Epoch 2/7


...

Validation Loss: 0.0097
Epoch 7/7


Training: 100%|██████████| 178/178 [00:09<00:00, 19.77it/s]


Training Loss: 0.0036


Validating: 100%|██████████| 23/23 [00:00<00:00, 88.89it/s]

Validation Loss: 0.0088


In [ ]:
def test_model(model, test_loader, device):
    model.eval()
    predictions, true_values = [], []
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Testing"):
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask).squeeze()
            predictions.extend(outputs.cpu().numpy())
            true_values.extend(labels.cpu().numpy())
    return predictions, true_values

predictions, true_values = test_model(model, test_loader, device)


for pred, true in zip(predictions[:10], true_values[:10]):
    print(f"Predicted: {pred:.2f}, Actual: {true:.2f}")


Testing: 100%|██████████| 23/23 [00:00<00:00, 82.56it/s]

Predicted: 0.90, Actual: 0.90
Predicted: 0.18, Actual: 0.20
Predicted: 0.81, Actual: 0.80
Predicted: 1.01, Actual: 1.00
Predicted: 0.43, Actual: 0.40
Predicted: 0.90, Actual: 1.00
Predicted: 0.33, Actual: 0.30
Predicted: 1.00, Actual: 1.00
Predicted: 0.66, Actual: 0.50
Predicted: 0.18, Actual: 0.10


In [ ]:
from transformers import AutoModel
import torch
import torch.nn as nn

# Define EmotionRegressor 
class EmotionRegressor(nn.Module):
    def __init__(self, model_name):
        super(EmotionRegressor, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.regressor = nn.Linear(self.bert.config.hidden_size, 1)
        self.dropout = nn.Dropout(0.1)

    def forward(self, input_ids, attention_mask, token_type_ids=None, **kwargs):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.regressor(pooled_output)
        return logits


def save_model(model, path):
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")


def load_model(model, path, device):
    model.load_state_dict(torch.load(path, map_location=device))
    print(f"Model loaded from {path}")
    return model


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model_name = "bert-base-uncased"
model = EmotionRegressor(model_name).to(device)


model_path = "emotion_regressor.pth"


save_model(model, model_path)


loaded_model = EmotionRegressor(model_name).to(device)
loaded_model = load_model(loaded_model, model_path, device)


loaded_model.eval()

example_input_ids = torch.tensor([[101, 2054, 2003, 2115, 2171, 102]]).to(device)
example_attention_mask = torch.tensor([[1, 1, 1, 1, 1, 1]]).to(device)
example_output = loaded_model(input_ids=example_input_ids, attention_mask=example_attention_mask)
print(f"Example output: {example_output}")


Model saved to emotion_regressor.pth


<ipython-input-33-6a1a8170a022>:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path, map_location=device))


Model loaded from emotion_regressor.pth
Example output: tensor([[-0.1041]], device='cuda:0', grad_fn=<AddmmBackward0>)


## Step 3 Doing emotion level change

In [ ]:

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm
import logging
import os
!pip install transformers torch tqdm


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cuda


In [ ]:
import pandas as pd

data = pd.read_csv('/content/drive/My Drive/combined_emotions.csv')

data['expected_intensity'] = data['intensity']


In [ ]:
def format_input(row):
    return f"adjust: {row['emotion']} | intensity: {row['expected_intensity']} | scenario: {row['scenario']}"

data['formatted_input'] = data.apply(format_input, axis=1)


In [ ]:
from sklearn.model_selection import train_test_split

train, temp = train_test_split(data, test_size=0.3, random_state=42)
val, test = train_test_split(temp, test_size=0.5, random_state=42)


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset

class EmotionDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=128):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        inputs = self.tokenizer.encode_plus(
            row['formatted_input'],
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        same_scenario = self.dataframe[self.dataframe['scenario'] == row['scenario']]
        target_row = same_scenario[same_scenario['intensity'] == row['expected_intensity']].iloc[0]
        target = self.tokenizer.encode(
            target_row['text'],
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )
        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'labels': target.squeeze()
        }

train_dataset = EmotionDataset(train, tokenizer)
val_dataset = EmotionDataset(val, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)


In [ ]:
from transformers import AdamW

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from tqdm import tqdm

def train_epoch(model, loader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in tqdm(loader, desc="Training"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(loader)

def validate_epoch(model, loader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in tqdm(loader, desc="Validating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            total_loss += outputs.loss.item()
    return total_loss / len(loader)


In [ ]:
epochs = 7
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    train_loss = train_epoch(model, train_loader, optimizer, device)
    print(f"Training Loss: {train_loss:.4f}")

    val_loss = validate_epoch(model, val_loader, device)
    print(f"Validation Loss: {val_loss:.4f}")

model.save_pretrained("t5_emotion_adjuster")
tokenizer.save_pretrained("t5_emotion_adjuster")


Epoch 1/7


Training: 100%|██████████| 312/312 [00:25<00:00, 12.21it/s]


Training Loss: 1.2869


Validating: 100%|██████████| 67/67 [00:02<00:00, 28.77it/s]


Validation Loss: 0.5334
Epoch 2/7


...

Training: 100%|██████████| 312/312 [00:24<00:00, 12.91it/s]


Training Loss: 0.3030


Validating: 100%|██████████| 67/67 [00:02<00:00, 29.66it/s]


Validation Loss: 0.2769


('t5_emotion_adjuster/tokenizer_config.json',
 't5_emotion_adjuster/special_tokens_map.json',
 't5_emotion_adjuster/spiece.model',
 't5_emotion_adjuster/added_tokens.json')

In [ ]:
test_dataset = EmotionDataset(test, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=8)

model.eval()
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Testing"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=128)

        decoded_inputs = tokenizer.batch_decode(input_ids, skip_special_tokens=True)
        decoded_outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

        for i in range(len(decoded_inputs)):
            print(f"Original: {decoded_inputs[i]}")
            print(f"Generated: {decoded_outputs[i]}")
            print("=" * 50)


Testing:   1%|▏         | 1/67 [00:00<00:26,  2.47it/s]

Original: adjust: sadness | intensity: 7 | scenario: I had to cancel plans I was really looking forward to
Generated: I felt a little nervous about how much I needed to do.
Original: adjust: caring | intensity: 9 | scenario: Your friend Rachel is struggling with anxiety attacks
Generated: Your anxiety and anxiety are my biggest concern - I'm here to help you overcome this anxiety.
Original: adjust: caring | intensity: 8 | scenario: Your coworker is struggling after losing their home in a fire
Generated: I’m apprehensive about how much they’re struggling.
Original: adjust: confusion | intensity: 1 | scenario: You're figuring out if someone's flirting or just being nice
Generated: I'm apprehensive about how they're flirting.
Original: adjust: fear | intensity: 3 | scenario: Experiencing an earthquake while inside a building
Generated: I’m apprehensive about how they’re going to be.
Original: adjust: joy | intensity: 2 | scenario: I took a perfect photo of a beautiful moment
Generated: I 

Testing:   3%|▎         | 2/67 [00:00<00:23,  2.75it/s]

Original: adjust: confusion | intensity: 10 | scenario: You're assembling a child's toy
Generated: I'm apprehensive about how they're doing.
Original: adjust: confusion | intensity: 5 | scenario: You're analyzing ancient astronomical records
Generated: I'm apprehensive about how ancient history is.
Original: adjust: caring | intensity: 10 | scenario: Your autistic child is being transferred to a new school
Generated: I’m apprehensive about how they’re going to be.
Original: adjust: anger | intensity: 9 | scenario: Receiving unfair criticism from a boss
Generated: I’m apprehensive about how unfair it is.
Original: adjust: confusion | intensity: 6 | scenario: You're dealing with passive-aggressive coworkers
Generated: I'm apprehensive about how they're doing.
Original: adjust: anger | intensity: 10 | scenario: A computer crashing during important work
Generated: I’m apprehensive about how much I’ll do.
Original: adjust: confusion | intensity: 7 | scenario: You're learning origami techniq

Testing:   4%|▍         | 3/67 [00:01<00:23,  2.75it/s]

...

Original: adjust: anger | intensity: 6 | scenario: A delayed package without an explanation
Generated: I’m apprehensive about the delay.
Original: adjust: sadness | intensity: 9 | scenario: I learned a sad story about someone’s past
Generated: I felt a little sad thinking about how they’d gone.
Original: adjust: anger | intensity: 9 | scenario: A promised callback not happening
Generated: I’m apprehensive about how they’re going to be.
Original: adjust: anger | intensity: 7 | scenario: Receiving vague or contradictory instructions
Generated: I’m apprehensive about how they’re going to be.
Original: adjust: joy | intensity: 9 | scenario: I got a promotion at work
Generated: I felt a little nervous about how much I needed to do.
Original: adjust: sadness | intensity: 4 | scenario: I missed an important deadline that I couldn’t make up
Generated: I felt a small sense of sadness, feeling like I missed something.
Original: adjust: joy | intensity: 6 | scenario: I saw someone’s eyes light up

Testing:  99%|█████████▊| 66/67 [00:22<00:00,  2.82it/s]

Original: adjust: caring | intensity: 8 | scenario: Your young colleague is experiencing workplace discrimination
Generated: I’m apprehensive about how they’re dealing with discrimination.
Original: adjust: confusion | intensity: 3 | scenario: You're analyzing political game theory
Generated: I'm apprehensive about how to solve this problem.
Original: adjust: caring | intensity: 10 | scenario: Your child's teacher is battling health issues while teaching
Generated: Your health and wellbeing are important to me - I'm here to help you stay safe and secure.
Original: adjust: caring | intensity: 1 | scenario: Your colleague is going through a messy custody battle
Generated: I’m apprehensive about how they’re going to fight for justice.
Original: adjust: confusion | intensity: 1 | scenario: You're studying dream symbolism
Generated: I'm apprehensive about how to make it.
Original: adjust: fear | intensity: 2 | scenario: Being chased by a wild animal while hiking
Generated: I’m apprehensive 

Testing: 100%|██████████| 67/67 [00:23<00:00,  2.88it/s]

Original: adjust: caring | intensity: 2 | scenario: Your pregnant colleague is working overtime
Generated: I’m apprehensive about how much time you’re working.
Original: adjust: sadness | intensity: 3 | scenario: I watched a sad documentary about a social issue
Generated: I felt a little sad thinking about how they’re dealing with this.
Original: adjust: anger | intensity: 5 | scenario: A toddler throwing a tantrum in public
Generated: I’m apprehensive about how they’re doing.
Original: adjust: anger | intensity: 8 | scenario: Cyberbullying targeting someone you care about
Generated: I’m apprehensive about how they’re dealing with cyberbullying.
Original: adjust: sadness | intensity: 9 | scenario: I had to let go of something I worked hard to achieve
Generated: I felt a little overwhelmed, feeling like I was doing something hard.
Original: adjust: sadness | intensity: 2 | scenario: I heard a song that reminded me of a difficult time
Generated: I felt a little agitated, feeling a little

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer_adjust = T5Tokenizer.from_pretrained('/content/t5_emotion_adjuster')
model_adjust = T5ForConditionalGeneration.from_pretrained('/content/t5_emotion_adjuster')
model_adjust.to(device)


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
def test_emotion_adjuster(input_text):
    encoded_input = tokenizer_adjust.encode(input_text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    with torch.no_grad():
        generated_ids = model_adjust.generate(encoded_input, max_length=512)

    adjusted_text = tokenizer_adjust.decode(generated_ids[0], skip_special_tokens=True)
    return adjusted_text


## Pipeline

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, T5Tokenizer, T5ForConditionalGeneration, BertModel

class CustomBertModel(torch.nn.Module):
    def __init__(self, model_name='bert-base-uncased'):
        super(CustomBertModel, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.dropout = torch.nn.Dropout(0.1)
        self.regressor = torch.nn.Linear(self.bert.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask, token_type_ids=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.regressor(pooled_output)
        return logits

tokenizer_classify = AutoTokenizer.from_pretrained("/content/emotion_classifier")
model_classify = AutoModelForSequenceClassification.from_pretrained("/content/emotion_classifier")

model_intensity = CustomBertModel()
model_state_dict = torch.load('/content/emotion_regressor.pth')
model_intensity.load_state_dict(model_state_dict)

tokenizer_adjust = T5Tokenizer.from_pretrained('/content/t5_emotion_adjuster')
model_adjust = T5ForConditionalGeneration.from_pretrained('/content/t5_emotion_adjuster')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_classify.to(device)
model_intensity.to(device)
model_adjust.to(device)

emotion_classes = {0: 'anger', 1: 'caring', 2: 'confusion', 3: 'fear', 4: 'joy', 5: 'sadness'}

def emotion_adjustment_pipeline(input_sentence, desired_intensity):
    inputs_classify = tokenizer_classify(input_sentence, return_tensors="pt", padding=True, truncation=True)
    inputs_classify = {key: val.to(device) for key, val in inputs_classify.items()}
    outputs_classify = model_classify(**inputs_classify)
    emotion_idx = torch.argmax(outputs_classify.logits, dim=1).item()
    emotion = emotion_classes[emotion_idx]

    outputs_intensity = model_intensity(**inputs_classify)
    current_intensity = torch.sigmoid(outputs_intensity.squeeze()).item()

    print(f"emotion of old sentence: {emotion}")
    print(f"intensity of old sentence: {current_intensity:.2f}")

    scaled_intensity = int(desired_intensity * 10) if 0 <= desired_intensity <= 1 else desired_intensity
    adjust_input = f"adjust: {emotion} | intensity: {scaled_intensity} | scenario: {input_sentence}"
    adjust_inputs = tokenizer_adjust(adjust_input, return_tensors="pt", padding=True, truncation=True, max_length=100)
    adjust_inputs = {key: val.to(device) for key, val in adjust_inputs.items()}
    generated_ids = model_adjust.generate(**adjust_inputs, max_length=100)
    adjusted_sentence = tokenizer_adjust.decode(generated_ids[0], skip_special_tokens=True)

    # Step 4:
    inputs_adjusted = tokenizer_classify(adjusted_sentence, return_tensors="pt", padding=True, truncation=True)
    inputs_adjusted = {key: val.to(device) for key, val in inputs_adjusted.items()}
    outputs_new_intensity = model_intensity(**inputs_adjusted)
    new_intensity = torch.sigmoid(outputs_new_intensity.squeeze()).item()


    print(f"new setence: {adjusted_sentence}")
    print(f"intensity of new sentence: {new_intensity:.2f}")

    return emotion, current_intensity, adjusted_sentence, new_intensity

# use pipeline
input_sentence = "I cried."
desired_intensity = 0.2
emotion, current_intensity, adjusted_sentence, new_intensity = emotion_adjustment_pipeline(input_sentence, desired_intensity)


<ipython-input-46-a2f43a6a48b2>:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load('/content/emotion_regressor.pth')


emotion of old sentence: sadness
intensity of old sentence: 0.40
new setence: I felt a little agitated, feeling a little apprehensive.
intensity of new sentence: 0.51


In [ ]:
def test_emotion_intensity_model(model, tokenizer, text_samples, device):
    model.eval()
    results = {}
    for text in text_samples:
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
        inputs = {key: val.to(device) for key, val in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
            intensity = torch.sigmoid(outputs.squeeze()).item()
        results[text] = intensity
    return results

# example
text_samples = [
    "I feel great about the progress I have made.",
    "I am so sad to hear about your loss.",
    "I am furious about the situation.",
    "I got so nervous that I want to died right now!",
    "I'm feeling a bit confused by the instructions."
]

# function
intensity_results = test_emotion_intensity_model(model_intensity, tokenizer_classify, text_samples, device)
for text, intensity in intensity_results.items():
    print(f"Text: {text}\nPredicted Intensity: {intensity:.2f}\n")


Text: I feel great about the progress I have made.
Predicted Intensity: 0.47

Text: I am so sad to hear about your loss.
Predicted Intensity: 0.41

Text: I am furious about the situation.
Predicted Intensity: 0.49

Text: I got so nervous that I want to died right now!
Predicted Intensity: 0.46

Text: I'm feeling a bit confused by the instructions.
Predicted Intensity: 0.50



In [ ]:
def test_emotion_adjuster(model, tokenizer, scenarios, device):
    model.eval()
    results = {}
    for scenario, emotion, intensity in scenarios:
        input_text = f"adjust: {emotion} | intensity: {intensity} | scenario: {scenario}"
        encoded_input = tokenizer.encode(input_text, return_tensors="pt", padding=True, truncation=True, max_length=100).to(device)
        with torch.no_grad():
            generated_ids = model.generate(encoded_input, max_length=100)
        adjusted_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
        results[input_text] = adjusted_text
    return results

scenarios = [
    ("I just got a new puppy and it's adorable.", "joy", 10),
    ("I just got a new puppy and it's adorable.", "joy", 1),
    ("I am very sad about the news.", "sadness", 10),
    ("I am very sad about the news.", "sadness", 1),
    ("This is the worst day ever.", "anger", 10)
]

adjustment_results = test_emotion_adjuster(model_adjust, tokenizer_adjust, scenarios, device)
for scenario, adjusted_text in adjustment_results.items():
    print(f"Scenario: {scenario}\nAdjusted Text: {adjusted_text}\n")


Scenario: adjust: joy | intensity: 10 | scenario: I just got a new puppy and it's adorable.
Adjusted Text: I smiled slightly, feeling like I'm a little overwhelmed.

Scenario: adjust: joy | intensity: 1 | scenario: I just got a new puppy and it's adorable.
Adjusted Text: I smiled slightly, feeling like I'm a little overwhelmed.

Scenario: adjust: sadness | intensity: 10 | scenario: I am very sad about the news.
Adjusted Text: I feel a little sad that they didn’t know what they were doing.

Scenario: adjust: sadness | intensity: 1 | scenario: I am very sad about the news.
Adjusted Text: I feel a little sad that they didn’t know what they were doing.

Scenario: adjust: anger | intensity: 10 | scenario: This is the worst day ever.
Adjusted Text: I’m apprehensive about how we’re going to be.



## Gradio for demo

In [ ]:
!pip install gradio transformers torch

import gradio as gr
import torch
from torch import nn
from transformers import AutoModelForSequenceClassification, AutoTokenizer, T5ForConditionalGeneration, AutoModel
import os

# Step 3: define Model 2 - Emotion Intensity Regressor class
class EmotionRegressor(nn.Module):
    def __init__(self, model_name):
        super(EmotionRegressor, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.regressor = nn.Linear(self.bert.config.hidden_size, 1) 
        self.dropout = nn.Dropout(0.1)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.regressor(pooled_output)
        return logits

# Step 4: 
# Model 1: Emotion Classifier
tokenizer1 = AutoTokenizer.from_pretrained("/content/emotion_classifier", local_files_only=True)
model1 = AutoModelForSequenceClassification.from_pretrained("/content/emotion_classifier", local_files_only=True)
model1.eval()

# Model 2: Emotion Intensity Regressor 
tokenizer2 = AutoTokenizer.from_pretrained("bert-base-uncased", local_files_only=True)
model2 = EmotionRegressor("bert-base-uncased")
model2.load_state_dict(torch.load("/content/emotion_regressor.pth", map_location=torch.device('cpu')))
model2.eval()

# Model 3: Emotion Adjuster
tokenizer3 = AutoTokenizer.from_pretrained("/content/t5_emotion_adjuster", local_files_only=True)
model3 = T5ForConditionalGeneration.from_pretrained("/content/t5_emotion_adjuster", local_files_only=True)
model3.eval()

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model1.to(device)
model2.to(device)
model3.to(device)

# Step 5: 
# Model 1: emotion classification
def predict_emotion(sentence, tokenizer, model):
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
    inputs = {key: val.to(device) for key, val in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    predicted_class = torch.argmax(outputs.logits, dim=1).item()
    emotion_classes = {0: 'anger', 1: 'caring', 2: 'confusion', 3: 'fear', 4: 'joy', 5: 'sadness'}
    return emotion_classes[predicted_class]

# Model 2: emotion intensity regressor
def predict_intensity(sentence, tokenizer, model):
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
    inputs = {key: val.to(device) for key, val in inputs.items()}

    with torch.no_grad():
        outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])

    predicted_intensity = torch.sigmoid(outputs.squeeze()).item() * 10  # project to 1-10
    return round(predicted_intensity, 2)

# Model 3: adjust emotion
def adjust_emotion(sentence, emotion, expected_intensity, tokenizer, model):
    formatted_input = f"adjust: {emotion} | intensity: {expected_intensity} | scenario: {sentence}"
    inputs = tokenizer(formatted_input, return_tensors="pt", padding=True, truncation=True)
    inputs = {key: val.to(device) for key, val in inputs.items()}
    with torch.no_grad():
        generated_ids = model.generate(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=128)
    adjusted_sentence = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return adjusted_sentence

# Step 6: define Gradio
def emotion_pipeline(input_sentence, expected_intensity):
    # Step 1: Use Model 1 to predict emotion
    emotion = predict_emotion(input_sentence, tokenizer1, model1)

    # Step 2: Use Model 2 to predict intensity
    current_intensity = predict_intensity(input_sentence, tokenizer2, model2)

    # Step 3: Use Model 3 to adjust the intensity
    adjusted_sentence = adjust_emotion(input_sentence, emotion, expected_intensity, tokenizer3, model3)

    return f"Predicted Emotion: {emotion}", f"Predicted Intensity: {current_intensity}", f"Adjusted Sentence: {adjusted_sentence}"

# Step 7: use Gradio to create web interface
interface = gr.Interface(
    fn=emotion_pipeline,
    inputs=[
        gr.Textbox(lines=2, placeholder="Enter a sentence to adjust...", label="Input Sentence"),
        gr.Slider(minimum=1, maximum=10, value=5, label="Expected Intensity"),
    ],
    outputs=[
        gr.Textbox(label="Predicted Emotion"),
        gr.Textbox(label="Predicted Intensity"),
        gr.Textbox(label="Adjusted Sentence"),
    ],
    title="Emotion Adjuster",
    description="This tool takes a sentence, predicts its emotion and intensity, then adjusts the sentence to a target intensity level.",
)

# Step 8: start Gradio web interface
interface.launch()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 112.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


<ipython-input-49-057cb3e1a286>:35: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model2.load_state_dict(torch.load("/content/emotion_regressor.pth", map_location=torch.devi

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://527c9f290a10e5005b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
